In [1]:
import mxnet as mx
from mxnet import gluon, autograd, ndarray
import numpy as np

# Download the MNIST dataset, then create the training and test sets 
train_data = mx.gluon.data.DataLoader(mx.gluon.data.vision.MNIST(train=True, transform=lambda data, label: (data.astype(np.float32)/255, label)),
                                      batch_size=32, shuffle=True)
test_data = mx.gluon.data.DataLoader(mx.gluon.data.vision.MNIST(train=False, transform=lambda data, label: (data.astype(np.float32)/255, label)),
                                     batch_size=32, shuffle=False)
# Initialize the model
net = gluon.nn.Sequential()

# Define the model architecture
with net.name_scope():
    net.add(gluon.nn.Dense(10)) # Output layer, one for each number 0-9

net.collect_params().initialize(mx.init.Normal(sigma=0.05), ctx=mx.cpu())
softmax_cross_entropy = gluon.loss.SoftmaxCrossEntropyLoss()

trainer = gluon.Trainer(net.collect_params(), 'sgd', {'learning_rate': .1})

# Loop through several epochs and watch the model improve
epochs = 1
for e in range(epochs):
    for i, (data, label) in enumerate(train_data):
        data = data.as_in_context(mx.cpu()).reshape((-1, 784))
        label = label.as_in_context(mx.cpu())
        with autograd.record(): # Start recording the derivatives
            output = net(data) # the forward iteration
            loss = softmax_cross_entropy(output, label)
            loss.backward()
        trainer.step(data.shape[0])
        # Provide stats on the improvement of the model over each epoch
        curr_loss = ndarray.mean(loss).asscalar()
        break

In [2]:
import tvm
import nnvm
from tvm.contrib import util, graph_runtime as runtime
from tvm import rpc

In [16]:
sym = (mx.symbol.Variable('data1') == mx.symbol.Variable('data2'))

In [17]:
sym, params = nnvm.frontend.from_mxnet(sym)
graph, lib, params = nnvm.compiler.build(sym, 'llvm', shape={"data1": (1, 5), "data2": (1, 5)}, params=params)
remote = rpc.LocalSession()
module = runtime.create(graph, lib, ctx = remote.cpu(0))

In [30]:
module.set_input("data1", np.zeros((1, 5)).astype('int8'))
module.set_input("data2", np.zeros((1, 5)).astype('int8'))
module.set_input(**params)
module.run()

TVMError: [17:34:35] /home/tian/tvm/src/runtime/graph/graph_runtime.cc:103: Check failed: ret == 0 (-1 vs. 0) [17:34:35] /home/tian/tvm/src/runtime/ndarray.cc:157: Check failed: from_size == to_size (5 vs. 20) TVMArrayCopyFromTo: The size must exactly match

Stack trace returned 10 entries:
[bt] (0) /home/tian/.conda/envs/tvm/lib/python3.7/site-packages/tvm-0.5.dev0-py3.7-linux-x86_64.egg/tvm/libtvm.so(dmlc::StackTrace[abi:cxx11]()+0x5b) [0x7f7a204782bb]
[bt] (1) /home/tian/.conda/envs/tvm/lib/python3.7/site-packages/tvm-0.5.dev0-py3.7-linux-x86_64.egg/tvm/libtvm.so(tvm::runtime::NDArray::CopyFromTo(DLTensor*, DLTensor*, void*)+0x6e7) [0x7f7a20892867]
[bt] (2) /home/tian/.conda/envs/tvm/lib/python3.7/site-packages/tvm-0.5.dev0-py3.7-linux-x86_64.egg/tvm/libtvm.so(TVMArrayCopyFromTo+0x6) [0x7f7a20892fa6]
[bt] (3) /home/tian/.conda/envs/tvm/lib/python3.7/site-packages/tvm-0.5.dev0-py3.7-linux-x86_64.egg/tvm/libtvm.so(tvm::runtime::GraphRuntime::SetInput(int, DLTensor*)+0xfc) [0x7f7a208d3

In [29]:
module.get_output(0, tvm.nd.empty((1, 5), dtype='int8'))

TVMError: [17:34:23] /home/tian/tvm/src/runtime/graph/graph_runtime.cc:123: Check failed: ret == 0 (-1 vs. 0) [17:34:23] /home/tian/tvm/src/runtime/ndarray.cc:157: Check failed: from_size == to_size (20 vs. 5) TVMArrayCopyFromTo: The size must exactly match

Stack trace returned 10 entries:
[bt] (0) /home/tian/.conda/envs/tvm/lib/python3.7/site-packages/tvm-0.5.dev0-py3.7-linux-x86_64.egg/tvm/libtvm.so(dmlc::StackTrace[abi:cxx11]()+0x5b) [0x7f7a204782bb]
[bt] (1) /home/tian/.conda/envs/tvm/lib/python3.7/site-packages/tvm-0.5.dev0-py3.7-linux-x86_64.egg/tvm/libtvm.so(tvm::runtime::NDArray::CopyFromTo(DLTensor*, DLTensor*, void*)+0x6e7) [0x7f7a20892867]
[bt] (2) /home/tian/.conda/envs/tvm/lib/python3.7/site-packages/tvm-0.5.dev0-py3.7-linux-x86_64.egg/tvm/libtvm.so(TVMArrayCopyFromTo+0x6) [0x7f7a20892fa6]
[bt] (3) /home/tian/.conda/envs/tvm/lib/python3.7/site-packages/tvm-0.5.dev0-py3.7-linux-x86_64.egg/tvm/libtvm.so(tvm::runtime::GraphRuntime::GetOutput(int, DLTensor*)+0x11c) [0x7f7a208